In [ ]:
%%time
import importlib
import multiprocessing as mp
from math import ceil, log
from pathlib import Path

import farkle.run_tournament as rt
import mpmath
from scipy.stats import norm

# -----------------------------------------------------------------
# You almost never change these:

GLOBAL_SEED = 42
BASE_OUT = Path("data/results")
JOBS     = None            # None → use all logical cores
BASE_OUT.mkdir(parents=True, exist_ok=True)

# Power-analysis helpers  ───────────────────────────────────────────

def harmonic_number(m: int) -> float:
    """Approximate H_m for m ≥ 1 (no mpmath needed)."""
    if m < 50_000:
        return sum(1/i for i in range(1, m+1))
    # Euler–Mascheroni expansion (error < 1e-7 for m ≥ 50 000)
    euler_gamma = mpmath.euler
    return log(m) + euler_gamma + 1/(2*m) - 1/(12*m**2)

def games_for_power(m: int,
                    n_players: int,
                    delta: float = 0.03,
                    power: float = 0.80,
                    q_fdr: float = 0.05) -> int:
    """
    Returns required *games per strategy* under BH-controlled FDR.
    One game per strategy per shuffle ⇒ result == num. shuffles.
    """
    # BH per-test alpha*  ≈  q / H_m   (Benjamini & Hochberg, 1995)
    alpha_star = q_fdr / harmonic_number(m)
    z_alpha = norm.isf(alpha_star / 2)          # two-sided
    z_beta  = norm.isf(1 - power)
    
    p0 = 1 / n_players if n_players > 2 else 0.5   # null win rate
    p1 = p0 + delta
    var  = p0*(1-p0) + p1*(1-p1)
    
    n = ((z_alpha + z_beta)**2 * var) / (delta**2)
    return ceil(n)

# CONFIG N-player run ───────────────────────────────────────────────
GRID        = 8_160         # strategies
N_PLAYERS   = 3
DELTA       = 0.03
POWER       = 0.80
Q_FDR       = 0.05
# ------------------------------------------------------------------

NUM_SHUFFLES        = games_for_power(GRID, N_PLAYERS,
                                      delta=DELTA,
                                      power=POWER,
                                      q_fdr=Q_FDR)
GAMES_PER_SHUFFLE   = GRID // N_PLAYERS
TOTAL_GAMES         = NUM_SHUFFLES * GAMES_PER_SHUFFLE

print(f"Benjamini–Hochberg FDR = {Q_FDR}")
print(f"  Strategies            : {GRID:,}")
print(f"  Players per table     : {N_PLAYERS}")
print(f"  Δ (detectable lift)   : {DELTA:.0%}")
print(f"  Power (1-β)           : {POWER:.0%}")
print(f"  Per-test α*           : {Q_FDR/harmonic_number(GRID):.4f}")
print(f"  Num. shuffles         : {NUM_SHUFFLES:,}")
print(f"  Games per shuffle     : {GAMES_PER_SHUFFLE}")
print(f"  Total games           : {TOTAL_GAMES:,}")



# ── patch driver-side constants ───────────────────────────────────
rt = importlib.reload(rt)                 # fresh copy each run
rt.N_PLAYERS         = N_PLAYERS
rt.GAMES_PER_SHUFFLE = GAMES_PER_SHUFFLE  # GRID // N_PLAYERS
rt.NUM_SHUFFLES      = NUM_SHUFFLES       # ← NEW: power-target cap

# ── output paths etc. (assumes BASE_OUT / GLOBAL_SEED / JOBS exist) ─
out_dir = BASE_OUT / f"{N_PLAYERS}_players"
out_dir.mkdir(exist_ok=True)
chkpt   = out_dir / "checkpoint.pkl"
row_dir = out_dir / "rows"

# ── launch ────────────────────────────────────────────────────────
mp.set_start_method("spawn", force=True)  # Windows & Jupyter safe
rt.run_tournament(
    global_seed         = GLOBAL_SEED,
    checkpoint_path     = chkpt,
    n_jobs              = JOBS,
    collect_metrics     = True,
    row_output_directory= row_dir,
)
print(f"✔ finished {N_PLAYERS}-player run →", out_dir)



In [ ]:
import pickle

import pandas as pd

# aggregated win / score sums
with open(chkpt, "rb") as fh: # type: ignore   chkpt defined in previous cell
    summary = pickle.load(fh)
summary_df = pd.DataFrame(summary)    # one row per strategy
summary_df.sort_values("wins", ascending=False)

# full per-game rows (optional; can be large)
rows = pd.read_parquet(row_dir) # type: ignore   row_dir defined in previous cell, auto-globs *.parquet
rows.head()


KeyError: 'wins'

In [ ]:
from pathlib import Path

# -----------------------------------------------------------------
# GLOBAL constants you rarely change
GLOBAL_SEED  = 42


BASE_OUT     = Path("data/results") 
BASE_OUT.mkdir(parents=True, exist_ok=True)
JOBS         = None                  # None → use all logical cores
# -----------------------------------------------------------------

# ── Power-analysis helpers ─────────────────────────────────────────

def shuffles_required(n_players: int,
                      delta: float = 0.03,
                      power: float = 0.90,
                      q_fdr: float = 0.02,
                      two_sided: bool = True) -> int:
    """
    Minimum *shuffles* (games/strategy) so that a two-proportion z-test
    has `power` to detect an absolute lift `delta`, while the Benjamini–
    Hochberg procedure controls FDR at `q_fdr`.
    
    One shuffle gives exactly one observation per strategy, so
    shuffles == observations/strategy.
    """
    p0      = 1 / n_players            # null win probability
    z_alpha = norm.isf((q_fdr/2) if two_sided else q_fdr) # type: ignore   imported in previous cell
    z_beta  = norm.isf(power)          # power = Φ(z_beta) # type: ignore   imported in previous cell
    
    var = p0*(1-p0) + (p0+delta)*(1-p0-delta)
    n   = ((z_alpha + z_beta)**2 * var) / (delta**2)
    return ceil(n) # type: ignore   imported in previous cell

# ── CONFIG: choose the table size you’re about to run ──────────────
GRID        = 8_160            # total strategies (constant)
N_PLAYERS   = 3                # ← change 2,3,4,… per run
DELTA       = 0.03
POWER       = 0.90
Q_FDR       = 0.02
# ------------------------------------------------------------------

NUM_SHUFFLES        = shuffles_required(N_PLAYERS,
                                        delta  = DELTA,
                                        power  = POWER,
                                        q_fdr  = Q_FDR)
GAMES_PER_SHUFFLE   = GRID // N_PLAYERS
TOTAL_GAMES         = NUM_SHUFFLES * GAMES_PER_SHUFFLE

print(f"Benjamini–Hochberg FDR (two-sided) : Q = {Q_FDR}")
print(f"Players per table                  : {N_PLAYERS}")
print(f"Detectable lift Δ                  : {DELTA:.0%}")
print(f"Power (1-β)                        : {POWER:.0%}")
print(f"Critical z_α                       : {norm.isf(Q_FDR/2):.3f}") # type: ignore   imported in previous cell
print(f"Shuffles (obs/strategy)            : {NUM_SHUFFLES:,}")
print(f"Games per shuffle                  : {GAMES_PER_SHUFFLE}")
print(f"Total games this run               : {TOTAL_GAMES:,}")

# -----------------------------------------------------------------
#   Launch the tournament in next cell
# -----------------------------------------------------------------


Benjamini–Hochberg FDR (two-sided) : Q = 0.02
Players per table                  : 3
Detectable lift Δ                  : 3%
Power (1-β)                        : 90%
Critical z_α                       : 2.326
Shuffles (obs/strategy)            : 551
Games per shuffle                  : 2720
Total games this run               : 1,498,720


In [ ]:
%%time
import importlib
import multiprocessing as mp

import farkle.run_tournament as rt

rt = importlib.reload(rt)                   # fresh copy each run
rt.N_PLAYERS         = N_PLAYERS
rt.GAMES_PER_SHUFFLE = GAMES_PER_SHUFFLE
rt.NUM_SHUFFLES      = NUM_SHUFFLES         # ← power-based cap

out_dir = BASE_OUT / f"{N_PLAYERS}_players"
out_dir.mkdir(exist_ok=True)
chkpt   = out_dir / "checkpoint.pkl"
row_dir = out_dir / "rows"

mp.set_start_method("spawn", force=True)    # Windows & Jupyter safe
rt.run_tournament(
    global_seed          = GLOBAL_SEED,
    checkpoint_path      = chkpt,
    n_jobs               = JOBS,
    collect_metrics      = True,
    row_output_directory = row_dir,
)
print(f"✔ finished {N_PLAYERS}-player run → {out_dir}")

16:35:34 checkpoint … 1/551 chunks, 1632 games
16:36:04 checkpoint … 63/551 chunks, 102816 games
16:36:35 checkpoint … 98/551 chunks, 159936 games
16:37:07 checkpoint … 138/551 chunks, 225216 games
16:37:40 checkpoint … 178/551 chunks, 290496 games
16:38:10 checkpoint … 219/551 chunks, 357408 games
16:38:41 checkpoint … 256/551 chunks, 417792 games
16:39:11 checkpoint … 292/551 chunks, 476544 games
16:39:41 checkpoint … 333/551 chunks, 543456 games
16:40:11 checkpoint … 367/551 chunks, 598944 games
16:40:41 checkpoint … 405/551 chunks, 660960 games
16:41:12 checkpoint … 444/551 chunks, 724608 games
16:41:43 checkpoint … 480/551 chunks, 783360 games
16:42:14 checkpoint … 522/551 chunks, 851904 games
16:42:35 finished - 899232 games


✔ finished 3-player run → data\results\3_players
CPU times: total: 12.5 s
Wall time: 7min 47s
